<a href="https://colab.research.google.com/github/ny052/StoreMapping/blob/main/sabae_shops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#seleniumとchromedriverをインストールしてヘッドレスクロームを準備する
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

#Googleドライブに接続
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 911kB 5.6MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
#モジュールのインポート
import sys
import os
import shutil
import csv
import re
import pandas as pd
from time import sleep
import datetime
import traceback
import openpyxl
from tqdm import tqdm
from openpyxl.styles.borders import Border, Side
import requests
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0')

base_url = 'https://google.com'
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(base_url)
print(driver.find_element(By.TAG_NAME, 'html').text)

About
Store
GmailImages
Sign in
Advertising
Business
How Search works
Carbon neutral since 2007
Privacy
Terms
Settings


In [ ]:
import pandas as pd

# データフレームを読み込む
df = pd.read_csv("drive/My Drive/Colab Notebooks/ColabPython/FromBooks/898.csv")

print(len(df))
print(df.columns.values)

72
['店舗名(日本語)' '店舗名(英語)' 'カテゴリー' '郵便番号' '都道府県名' '市区町村名' '行政区名' '住所' '緯度' '経度'
 '電話番号' 'webサイト' '営業開始時間' '営業終了時間' '営業時間' '定休日' '駐車可能台数' '男子トイレの数'
 '女子トイレの数' '男女兼用トイレの数' '説明(日本語)' '説明(英語)' '更新日' '登録日' '自治体コード']


In [ ]:
# 店舗のある地点（緯度、経度）と店舗名をリスト化する
store = df[["経度","緯度","店舗名(日本語)"]].values
print(len(store))
print(store)

72
[[136.184073 35.956477 '鯖江市役所']
 [136.183866 35.938552 '地域事業主 わどう']
 [136.18510700000002 35.945617999999996 '佐野蕎麦 塩だけで食べる異次元そば']
 [136.18717900000001 35.946157 'ヨーロッパン キムラヤ']
 [136.185571 35.946548 'ミート＆デリカささき']
 [136.30000900000002 35.948063 'かわだ温泉 ラポーゼかわだ']
 [136.30000900000002 35.948063 'ラポーゼかわだ ほたるの里']
 [136.181307 35.967237 'リラクゼーション整体 癒しのたまご']
 [136.1781753 35.946884999999995 'コスメティックス美香']
 [136.177074 35.950264000000004 'green parlour ベルベール']
 [136.212288 35.933178999999996 '聴琴亭']
 [136.185026 35.946248 'たんなん夢レディオ']
 [136.19558600000002 35.968798 '菓子処安価堂']
 [136.14443630000002 35.964309 '表具処 紅屋紅陽堂']
 [136.187874 35.943071 '甘味茶屋\u3000極']
 [136.185926 35.94511 '亀屋萬年堂']
 [136.164807 35.948256 'とくはし整骨院']
 [136.183602 35.94506 '株式会社内田工務店']
 [136.149706 35.944437 '林呉服店']
 [136.185429 35.949481 '酒のあかし']
 [136.186115 35.945423999999996 'エルザスコーヒー']
 [136.187757 35.943571 '久保田酒店']
 [136.181271 35.947309000000004 'イル・ヴィゴーレ']
 [136.181271 35.947309000000004 'メゾン・ド・レトワール']
 [136.198841999

In [ ]:
import pandas as pd
import folium

# データフレームを読み込む
df = pd.read_csv("drive/My Drive/Colab Notebooks/ColabPython/FromBooks/898.csv")

# 店舗のある地点（緯度、経度）と店舗名をリスト化する
store = df[["経度","緯度","店舗名(日本語)"]].values

# 地図を作って書き出す
m = folium.Map(location=[35.942957, 136.198863], zoom_start=16)
for data in store:
    folium.Marker([data[1], data[0]], tooltip=data[2]).add_to(m)
m